In [2]:
import os
import skimage.io as io
import skimage.transform as trans
import random
from keras import metrics
from keras import backend as K
from keras.initializers import *
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

Using TensorFlow backend.


In [3]:
random.seed('beachclassification')
model_initializer = glorot_uniform
kernel_init = model_initializer(seed=random.randint(0,5))
model_learning_rate = 1e-3

In [4]:
def sorensen_dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    coef = (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return coef

def sorensen_dice_coef_loss(y_true, y_pred):
    return 1 - sorensen_dice_coef(y_true, y_pred)

In [5]:
def generateUnet():
    inputs = Input(shape=(256,256,4))
    conv1 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(inputs)
    conv1 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(pool1)
    conv2 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(pool2)
    conv3 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    
    conv4 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(pool3)
    conv4 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv4)
    drop4 = Dropout(0.01)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(drop4)
    
    conv5 = Conv2D(1024, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(pool4)
    conv5 = Conv2D(1024, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv5)
    drop5 = Dropout(0.01)(conv5)
    
    up6 = concatenate([Conv2DTranspose(512, (2,2), strides=(2,2), padding='same', kernel_initializer=kernel_init)(drop5), drop4], axis=3)
    conv6 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(up6)
    conv6 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv6)
    
    up7 = concatenate([Conv2DTranspose(256, (2,2), strides=(2,2), padding='same', kernel_initializer=kernel_init)(conv6), conv3], axis=3)
    conv7 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(up7)
    conv7 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv7)
    
    up8 = concatenate([Conv2DTranspose(128, (2,2), strides=(2,2), padding='same', kernel_initializer=kernel_init)(conv7), conv2], axis=3)
    conv8 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(up8)
    conv8 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv8)
    
    up9 = concatenate([Conv2DTranspose(64, (2,2), strides=(2,2), padding='same', kernel_initializer=kernel_init)(conv8), conv1], axis=3)
    conv9 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(up9)
    conv9 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv9)
    conv9 = Conv2D(2, (3,3), activation='relu', padding='same', kernel_initializer=kernel_init)(conv9)
    
    conv10 = Conv2D(1, (1,1), activation='softmax')(conv9)
    
    model = Model(inputs=[inputs], outputs=[conv10])
    
    model.compile(optimizer=Adam(lr=model_learning_rate), loss=sorensen_dice_coef_loss, metrics=[sorensen_dice_coef])
    
    return model

In [6]:
model = generateUnet()

In [7]:
K.image_data_format()

'channels_last'

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 4)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 2368        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [ ]:
#class ImageData():
    
    #def __init__(self, image_dir, label_dir):
        #self.image = []
        #self.label = []
        #for file in os.listdir(image_dir):
            #if file.endswith(".tif"):
                #self.image.append(io.imread(os.path.join(image_dir, file)))
        #for file in os.listdir(label_dir):
        #    if file.endswith(".tif"):
        #        self.label.append(io.imread(os.path.join(label_dir, file)))
    
    #def getAllImages(self):
        #return np.array(self.image)
    
    #def getAllLabels(self):
        #return np.array(self.label)

In [9]:
def getImageryInRange(range_of_ids):
    images = []
    for val in range(range_of_ids[0], range_of_ids[1]):
        path = "/home/paperspace/ciaran/data/train_normalised/blue_anchor_" + str(val) + ".tif"
        images.append(io.imread(path))
    return np.array(images)
def getLabelsInRange(range_of_ids):
    labels = []
    for val in range(range_of_ids[0], range_of_ids[1]):
        path = "/home/paperspace/ciaran/data/label/blue_anchor_" + str(val) + ".tif"
        labels.append(io.imread(path))
    return np.array(labels)

In [10]:
training_images = getImageryInRange((3000, 6000))
training_labels = getLabelsInRange((3000, 6000))

In [ ]:
#images = ImageData("/home/paperspace/ciaran/data/train_normalised/",
#                   "/home/paperspace/ciaran/data/label/")
#all_images = images.getAllImages()
#training_images = all_images[((40 * len(all_images))//100):((60 * len(all_images))//100)]
#all_labels = images.getAllLabels()
#training_labels = all_labels[((40 * len(all_labels))//100):((60 * len(all_labels))//100)]

In [11]:
import h5py
import re
from importlib import reload
from keras.callbacks import EarlyStopping, TensorBoard
from keras.models import load_model
from keras.models import save_model

In [12]:
t_board = TensorBoard(log_dir="./models/lr-1e-3_p-30_e-200_bs-16/logs", histogram_freq=0, write_grads=False, write_graph=True, write_images=False)
stop_early = EarlyStopping(patience=30)
model.fit(training_images, training_labels, validation_split=0.2, epochs=200, batch_size=16, shuffle=True, callbacks=[stop_early, t_board])
model.save("./models/lr-1e-3_p-30_e-200_bs-16/model")

Train on 2400 samples, validate on 600 samples
Epoch 1/200
2400/2400 [==============================] - 138s 57ms/step - loss: 0.5579 - sorensen_dice_coef: 0.4421 - val_loss: 0.5251 - val_sorensen_dice_coef: 0.4749
Epoch 2/200
2400/2400 [==============================] - 129s 54ms/step - loss: 0.5588 - sorensen_dice_coef: 0.4412 - val_loss: 0.5251 - val_sorensen_dice_coef: 0.4749
Epoch 3/200
2400/2400 [==============================] - 129s 54ms/step - loss: 0.5575 - sorensen_dice_coef: 0.4425 - val_loss: 0.5251 - val_sorensen_dice_coef: 0.4749
Epoch 4/200
2400/2400 [==============================] - 130s 54ms/step - loss: 0.5611 - sorensen_dice_coef: 0.4389 - val_loss: 0.5251 - val_sorensen_dice_coef: 0.4749
Epoch 5/200
2400/2400 [==============================] - 130s 54ms/step - loss: 0.5573 - sorensen_dice_coef: 0.4427 - val_loss: 0.5251 - val_sorensen_dice_coef: 0.4749
Epoch 6/200
2400/2400 [==============================] - 130s 54ms/step - loss: 0.5579 - sorensen_dice_coef: 0.44

KeyboardInterrupt: 